In [1]:
import pyspark.sql.functions as F
from pyspark.sql.types import *
import pandas as pd
import numpy as np
import re
import os 

import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession
findspark.find()

spark = SparkSession.builder.getOrCreate()
from pyspark.sql.types import *

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/23 13:51:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns



In [2]:
# check
spark

In [3]:
os.getcwd()

'/Users/echo/Desktop/BigData/Group_Assignment'

# 1. Kickstarter Data

<font size="3">First, lets import the kickstarter data. This data was scraped by a webscraper and can be found online on https://webrobots.io/kickstarter-datasets/. Each month, data is scraped from kickstarter.com and is stored in json files. This means that for each monthly scrape new projects that were launched after the previous scrape will now appear in the data. Projects that were included in the previous scrape but that reached their deadline before the current scrape will no longer appear in the data. Projects that were included in the previous scrape but that did not yet reached their deadline will again be included in the 
    


## 1.1. Import json files

In [4]:
# define the data directory
data_dir = "./data/kickstarter/" # find on websit of https://webrobots.io/kickstarter-datasets/
# get all the kickstarter files
kickstarter_files = [os.path.join(data_dir, obs) for obs in os.listdir(data_dir)]

In [5]:
# check
kickstarter_files

['./data/kickstarter/Kickstarter_2020-03-12T03_20_06_551Z.json.gz',
 './data/kickstarter/Kickstarter_2020-04-16T03_20_04_541Z.json.gz',
 './data/kickstarter/Kickstarter_2020-05-14T03_20_08_560Z.json.gz',
 './data/kickstarter/Kickstarter_2020-06-18T03_20_07_487Z.json.gz',
 './data/kickstarter/Kickstarter_2020-07-16T03_20_08_086Z.json.gz',
 './data/kickstarter/Kickstarter_2020-08-13T03_20_17_470Z.json.gz',
 './data/kickstarter/Kickstarter_2020-09-17T03_20_18_143Z.json.gz',
 './data/kickstarter/Kickstarter_2020-10-15T03_20_03_128Z.json.gz',
 './data/kickstarter/Kickstarter_2020-11-12T03_20_11_453Z.json.gz',
 './data/kickstarter/Kickstarter_2020-12-17T03_20_12_051Z.json.gz',
 './data/kickstarter/Kickstarter_2021-01-14T03_20_05_328Z.json.gz',
 './data/kickstarter/Kickstarter_2021-02-11T03_20_07_976Z.json.gz',
 './data/kickstarter/Kickstarter_2021-03-18T03_20_11_507Z.json.gz',
 './data/kickstarter/Kickstarter_2021-04-15T03_20_08_451Z.json.gz',
 './data/kickstarter/Kickstarter_2021-05-13T03_2

In [6]:
# import data
kickstarter_df = spark.read.json(kickstarter_files)

In [7]:
# inspect number of observations
kickstarter_df.count()

3330989

In [8]:
kickstarter_df.printSchema()

root
 |-- data: struct (nullable = true)
 |    |-- backers_count: long (nullable = true)
 |    |-- blurb: string (nullable = true)
 |    |-- category: struct (nullable = true)
 |    |    |-- analytics_name: string (nullable = true)
 |    |    |-- color: long (nullable = true)
 |    |    |-- id: long (nullable = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- parent_id: long (nullable = true)
 |    |    |-- parent_name: string (nullable = true)
 |    |    |-- position: long (nullable = true)
 |    |    |-- slug: string (nullable = true)
 |    |    |-- urls: struct (nullable = true)
 |    |    |    |-- web: struct (nullable = true)
 |    |    |    |    |-- discover: string (nullable = true)
 |    |-- converted_pledged_amount: long (nullable = true)
 |    |-- country: string (nullable = true)
 |    |-- country_displayable_name: string (nullable = true)
 |    |-- created_at: long (nullable = true)
 |    |-- creator: struct (nullable = true)
 |    |    |-- avatar: struct 

## 1.2. Parse json files

<font size="3">Lets now parse the json files. This means that we will extract the relevant information from the json files such that for each project, we will have one row of features.

In [9]:
# select interesting features
kickstarter_sub = kickstarter_df.select(F.col("run_id"),
                                       F.col("data.id"),
                                       F.col("data.name").alias("project_name"),
                                       F.col("data.blurb").alias("project_description"),
                                       F.col("data.urls.web.project").alias("project_url"),
                                       F.col("data.photo.full").alias("photo_url"),
                                       F.col("data.country"),
                                       F.col("data.currency"),
                                       F.col("data.staff_pick"),
                                       F.col("data.creator.name").alias("creator_name"),
                                       F.col("data.backers_count"),
                                       F.col("data.goal"),
                                       F.col("data.pledged"),
                                       F.col("data.state"),
                                       F.col("data.launched_at"),
                                       F.col('data.category.analytics_name'),
                                       F.col('data.category.name'),
                                       F.col('data.category.parent_name'),
                                       F.col('data.category.slug'),
                                       F.col("data.deadline"))


## 1.3. Peprocess data

<font size="3">Now that the data is parsed, we can construct some more meaningfull features. 
We will also subset the data by obtaining only those projects that were launched after and ended before the start and end of our period of interest. 

In [10]:
# preprocess kickstarter data
kickstarter_processed = kickstarter_sub.withColumn("scrape_date", F.to_date(F.regexp_extract(F.col("run_id"), pattern="(.*)(\d{4}-\d{2}-\d{2})(.*)", idx=2))) \
                                       .withColumn("scrape_date", F.to_date(F.col("scrape_date"), format="yyyy-mm-dd")) \
                                       .withColumn("launched_at", F.from_unixtime(F.col("launched_at"))) \
                                       .withColumn("deadline", F.from_unixtime(F.col("deadline"))) \
                                       .withColumn("duration", F.datediff(F.col("deadline"), F.col("launched_at"))) \
                                       .filter(F.col("launched_at") >= F.lit("2020-01-01")) \
                                       .sort("project_url", "scrape_date")

In [11]:
# check number of observations
kickstarter_processed.count()

501295

In [12]:
# inspect first 20 observations
df_pd = kickstarter_processed.toPandas()

## Creating the dependent variable project succes

In [13]:
# remove live
kickstarter_lable_df = kickstarter_processed.filter(F.col("state") != "live") \
                                             .cache()

In [14]:
# convert dataframe to pandas
kickstarter_df = kickstarter_lable_df.toPandas()

In [15]:
# set scrape_date to date
kickstarter_df['scrape_date'] =  pd.to_datetime(kickstarter_df['scrape_date'])

# group id by max scrape date to get max scrape date per unique project
project_id_max_scrape = kickstarter_df.groupby('id').agg({'scrape_date':'max'}).reset_index()

# merge project id and max scrapte date back to original dataframe to get end state per project 
kickstarter_depedent = kickstarter_df.merge(project_id_max_scrape, on = ['id', 'scrape_date'], how = 'inner').drop_duplicates()


In [16]:
kickstarter_depedent.shape

(36965, 22)

In [17]:
# check for unique values per column in the dataset
for col in kickstarter_depedent.columns:
    print("%s has %s unique values."  %(col,len(kickstarter_depedent[col].unique())))

run_id has 19 unique values.
id has 36965 unique values.
project_name has 36931 unique values.
project_description has 36608 unique values.
project_url has 36965 unique values.
photo_url has 36943 unique values.
country has 25 unique values.
currency has 15 unique values.
staff_pick has 2 unique values.
creator_name has 29419 unique values.
backers_count has 2586 unique values.
goal has 2330 unique values.
pledged has 20747 unique values.
state has 3 unique values.
launched_at has 36834 unique values.
analytics_name has 161 unique values.
name has 160 unique values.
parent_name has 16 unique values.
slug has 170 unique values.
deadline has 33907 unique values.
scrape_date has 19 unique values.
duration has 78 unique values.


So now we got a dataframe outcome_projects with the unique combinations of id, state, max scrape_date, project descirption and project name. What we can conclude from above loop is that there are some project descriptions/names that occur multiple times and hence we have to use the id to evaluate the performance since id is unique.

In [18]:
kickstarter_depedent['project_name'].value_counts().head(10)

Blue                                                           3
Eden's Grove Launch (Canceled)                                 3
I AM                                                           3
Be Kind To Your Mind                                           2
"Los Influs" | Canciones infantiles para YouTube (Canceled)    2
Corail - The sneakers that clean up the sea (Canceled)         2
Sprout                                                         2
Wes's Falafel Burger (Canceled)                                2
The Office Enamel Pin Collection                               2
STiX CraftHouse                                                2
Name: project_name, dtype: int64

In [19]:
kickstarter_depedent['state'].value_counts(normalize=True)*100

successful    73.455972
failed        23.432977
canceled       3.111051
Name: state, dtype: float64

## Creating the basetable with the independent variables

In [20]:
# filter on live because each project starts live and hence only projects that are live can be first, so these are 
# the ones that we will include
kickstarter_basetable = kickstarter_processed.filter(F.col("state") == "live") \
                                            .cache()

# convert dataframe to pandas
kickstarter_basetable = kickstarter_basetable.toPandas()

# merge project id and min scrapte date back to original dataframe to get first scrape per project
kickstarter_basetable['scrape_date'] =  pd.to_datetime(kickstarter_basetable['scrape_date'])

In [21]:
# group id by min scrape date to get min scrape date per unique project so we can filter on the first 7 days (first scrape date)
project_id_min_scrape = kickstarter_basetable.groupby('id').agg({'scrape_date':'min'}).reset_index()

# rename for easy joining
project_id_min_scrape.columns = ['id','scrape_date']

# merge max scrape dat and dataframe
kickstarter_basetable = kickstarter_basetable.merge(project_id_min_scrape, on = ['id','scrape_date'], how = 'inner').drop_duplicates()



In [22]:
kickstarter_basetable.shape

(46119, 22)

In [23]:
# check for unique values per column in the dataset
# we can conclude that we still have some duplicates in the dataset
for col in kickstarter_basetable.columns:
    print("%s has %s unique values."  %(col,len(kickstarter_basetable[col].unique())))

run_id has 19 unique values.
id has 42721 unique values.
project_name has 42622 unique values.
project_description has 42385 unique values.
project_url has 42722 unique values.
photo_url has 42733 unique values.
country has 25 unique values.
currency has 15 unique values.
staff_pick has 2 unique values.
creator_name has 35880 unique values.
backers_count has 2239 unique values.
goal has 2914 unique values.
pledged has 19702 unique values.
state has 1 unique values.
launched_at has 42576 unique values.
analytics_name has 154 unique values.
name has 160 unique values.
parent_name has 16 unique values.
slug has 170 unique values.
deadline has 39454 unique values.
scrape_date has 19 unique values.
duration has 64 unique values.


In [24]:
# drop columns we don't need anymore
kickstarter_basetable = kickstarter_basetable.drop(columns=['state'])

What we can observe below is that a lot of projects that have the state live already have higher pledged than goal and hence these projects could actually already be considered as a successful project since the goal has already been reached. 

In [25]:
(kickstarter_basetable['pledged'] > kickstarter_basetable['goal']).sum()

16709

### What projects do we keep? We inner join so we only keep projects that went from live --> succes/fail/cancelled?

In [106]:
final_df = kickstarter_depedent[['state','id']].merge(kickstarter_basetable, how ='inner', on='id')

In [107]:
for column in final_df.columns:
    print(column, len(final_df[column].unique()))

state 3
id 28629
run_id 17
project_name 28614
project_description 28467
project_url 28629
photo_url 28641
country 25
currency 15
staff_pick 2
creator_name 24377
backers_count 2191
goal 2024
pledged 16990
launched_at 28544
analytics_name 139
name 160
parent_name 16
slug 170
deadline 26419
scrape_date 17
duration 62


So we have 28629 unique projects that have a live status and a status that is succesful/failed/cancelled in the period between 01/01/2020 and 01/09/2021

#### Add a column to indicate how many days the difference is between the scrape and the launched date

In [108]:
# Convert to datetime
final_df["launched_at"] = pd.to_datetime(pd.to_datetime(final_df["launched_at"], format = '%Y-%m-%d').dt.date)
final_df["scrape_date"] = pd.to_datetime(pd.to_datetime(final_df["scrape_date"], format = '%Y-%m-%d').dt.date)

# create days since scrape
final_df['Days_since_scrape'] = (final_df['scrape_date'] ) - final_df["launched_at"]


In [109]:
# select variables we will use for modeling
variables = ['state',
             'id',
             'project_name',
             'project_description',
             'project_url',
             'creator_name',
             'staff_pick',
             'goal',
             'launched_at',
             'analytics_name',
             'name',
             'parent_name',
             'slug',
             'scrape_date',
             'duration',
             'pledged',
             'backers_count',
             'Days_since_scrape'
            ]

In [110]:
final_df = final_df[variables].drop_duplicates()

## Filtering dataframe based on 7 days or 25 percent of project duration time 
Based on the information on the kickstarter database we observed that the goal, creator name, launched at and the duration cannot be changed during the project funding period. Hence, we will set all other variables after 7 days, or 25% of the duration period for projects with duration >= 30 to na, but will keep the other variables the same as these values cannot change. Hence, even though we do not have scrape within the independent period, no information after the independent period is used in the models since this information has not changed after the independent period.  


https://help.kickstarter.com/hc/en-us/articles/115005135314-Can-a-project-be-edited-after-launching-

In [111]:
# Create column that contains the days within the first 25% of the duration
final_df['Duration_25_perc'] = np.round(final_df['duration'] *0.25,0 )

In [112]:
# Select all the columns 
columns_to_na = [col for col in list(final_df.columns) if col not in ['goal','state','creator_name','launched_at','project_url']]

In [113]:
df_no_25_perc_limit = final_df[['goal','duration','id','state','creator_name','launched_at','Duration_25_perc','project_url']]

In [114]:
df_no_25_perc_limit.head()

,goal,duration,id,state,creator_name,launched_at,Duration_25_perc,project_url
0,500.0,30,1440011688,successful,Lithiumaniac,2021-04-08,8.0,https://www.kickstarter.com/projects/-aeolus/a...
1,2000.0,30,689293158,successful,CYON,2020-05-05,8.0,https://www.kickstarter.com/projects/-cyon-/cy...
2,18000.0,30,39986636,successful,Dam,2021-04-06,8.0,https://www.kickstarter.com/projects/-myowngam...
3,10000.0,30,520882120,successful,mystiic,2020-10-03,8.0,https://www.kickstarter.com/projects/-mystiic/...
4,6000.0,30,1347440454,successful,mystiic,2020-04-18,8.0,https://www.kickstarter.com/projects/-mystiic/...


In [115]:
df_25_perc_limit = final_df[columns_to_na]

In [116]:
# filter dataframe based on condition less than 7 days (<= because start at 0)
# filter duration more than 30 days on percentage duration 25% (add 1 to scrape date otherwise starts at 0 days)
df_25_perc_limit = df_25_perc_limit[ (df_25_perc_limit['Days_since_scrape'].dt.days <= 6 ) |
                  (
                      (df_25_perc_limit['duration'] >30) & (df_25_perc_limit['Duration_25_perc'] > df_25_perc_limit['Days_since_scrape'].dt.days )
                  )]

In [117]:
final_df = df_no_25_perc_limit.merge(df_25_perc_limit.drop(columns=['duration','Duration_25_perc']), on = 'id', how = 'left').drop_duplicates()

In [123]:
final_df['Days_since_scrape'].value_counts()

2 days     2300
1 days     1394
3 days     1189
6 days     1095
5 days      651
4 days      466
7 days      344
8 days      310
9 days      305
0 days      289
10 days     211
13 days     127
14 days     122
12 days     114
11 days     104
Name: Days_since_scrape, dtype: int64

In [124]:
# check if lauchned dat for projects with larger durations is within days since scrape limit
final_df[final_df['Days_since_scrape'].dt.days==14].head()

,goal,duration,id,state,creator_name,launched_at,Duration_25_perc,project_url,project_name,project_description,staff_pick,analytics_name,name,parent_name,slug,scrape_date,pledged,backers_count,Days_since_scrape
43,15000.0,60,1310701070,failed,Marlon Jones,2021-03-04,15.0,https://www.kickstarter.com/projects/101070927...,Sports World United the only sports social med...,Sports World United is the social media app st...,False,None,Apps,Technology,technology/apps,2021-03-18,2.00,2.0,14 days
185,23500.0,60,1674147714,failed,Julio Cesar Vargas Castro,2020-07-30,15.0,https://www.kickstarter.com/projects/107691688...,Mecanismo de apertura de puerta con pedal,"Mitiga el nivel de propagación del virus ""Covi...",False,None,Gadgets,Technology,technology/gadgets,2020-08-13,1.00,1.0,14 days
840,15000.0,60,33234742,canceled,Meranda,2021-04-01,15.0,https://www.kickstarter.com/projects/132325775...,Meranda's Micro bakery,Mobile micro bakery.,False,Farmer's Markets,Farmer's Markets,Food,food/farmer's markets,2021-04-15,176.00,5.0,14 days
1258,2500.0,61,477253221,failed,Carter Pembridge,2020-10-29,15.0,https://www.kickstarter.com/projects/149304461...,STAR WARS: Scars of the Past,STAR WARS Scars of the Past is an animated tv ...,False,None,Animation,Film & Video,film & video/animation,2020-11-12,251.29,3.0,14 days
1627,200.0,60,384170202,successful,The Family Project,2020-04-02,15.0,https://www.kickstarter.com/projects/163054470...,Dark Ice Saga: The Rescue,"A throwback-platformer for PC, designed by The...",False,None,Video Games,Games,games/video games,2020-04-16,48.00,6.0,14 days


There seems to be some error in the kickstarter data since for quite some scrapped kickstarter data the state = live, but, the amount pledged is already higher than the goal and hence these projects are actually already successful. This limits the managerial insights that can be derived from this variable. 

In [125]:
more_than = final_df[final_df['pledged'] > final_df['goal']]

In [126]:
more_than['state'].value_counts(normalize=True)*100

successful    99.624765
canceled       0.343965
failed         0.031270
Name: state, dtype: float64

### However, we still have duplicate values, this is due to the fact that some projects have multiple first scrapes that include different pledged amounts and backers

In [127]:
final_df.shape

(29785, 19)

In [128]:
final_df.groupby('id').agg({'pledged': 'count'}).reset_index().sort_values(by='pledged',ascending=False).head(2)

,id,pledged
25145,1883988190,2
14385,1077736380,2


In [129]:
final_df[final_df['id'] == 365762416]

,goal,duration,id,state,creator_name,launched_at,Duration_25_perc,project_url,project_name,project_description,staff_pick,analytics_name,name,parent_name,slug,scrape_date,pledged,backers_count,Days_since_scrape
5016,12000.0,60,365762416,failed,NiC (Aaron or A-A-Ron),2020-07-16,15.0,https://www.kickstarter.com/projects/acrossame...,NiC (upcoming artist),I made a song called StaY InsidE and wanted to...,False,None,Pop,Music,music/pop,2020-07-16,16.0,2.0,0 days
5017,12000.0,60,365762416,failed,NiC (Aaron or A-A-Ron),2020-07-16,15.0,https://www.kickstarter.com/projects/acrossame...,NiC (upcoming artist),I made a song called StaY InsidE and wanted to...,False,None,Pop,Music,music/pop,2020-07-16,21.0,3.0,0 days


### Hence, we caclulate the highest plegded per project id and inner join this back onto the dataframe to get the highest pleged/backers per project 

In [130]:
# calculate max pleged per project id in the first 7 days 
max_pledged = final_df.groupby('id').agg({'pledged': 'max'}).reset_index()

final_df = final_df.merge(max_pledged, on = ['id','pledged'], how = 'inner').drop_duplicates()

### After this we still have some very small amount of duplicates in the data as can be seen below. 
These duplicates, however, are some sort of error in the date since these scrapes are the same, but have different small diffeences in some of the variables. Since we can't really objectivly determine which one is correct, and the number of these project descriptions is very small, we don't think that this will influence our model and hence we decided to dump these projects from our dataset. 

In [131]:
final_df.shape

(28636, 19)

In [132]:
id_count = final_df['id'].value_counts().reset_index()
id_count.head(15)

,index,id
0,791111913,2
1,1927865946,2
2,1204314581,2
3,1065140923,2
4,1786627104,2
5,794056644,2
6,325745443,2
7,538013404,1
8,2107833154,1
9,1633390421,1


In [133]:
final_df[final_df['id'] == 1927865946]

,goal,duration,id,state,creator_name,launched_at,Duration_25_perc,project_url,project_name,project_description,staff_pick,analytics_name,name,parent_name,slug,scrape_date,pledged,backers_count,Days_since_scrape
968,30000.0,30,1927865946,failed,Victor Lee,2021-01-12,8.0,https://www.kickstarter.com/projects/147725364...,Wipe On Armor: Nanotechnology-based protection...,Screen Protector With Anti-microbial Propertie...,False,None,Gadgets,Technology,technology/gadgets,2021-01-14,1902.0,36.0,2 days
969,30000.0,30,1927865946,failed,Victor Lee,2021-01-12,8.0,https://www.kickstarter.com/projects/147725364...,Wipe On Armor: Nanotechnology-based protection...,"Wipe On Armor, a nanotechnology-based coating ...",False,None,Gadgets,Technology,technology/gadgets,2021-01-14,1902.0,36.0,2 days


In [134]:
# get the unique projects with unqiue values per row
unique_ids = id_count[id_count['id']==1]
unique_ids = unique_ids[['index']]
unique_ids = unique_ids.rename(columns={'index': 'id'})

# join this back on the final dataframe to keep the unique correct project descriptions only
final_df = final_df.merge(unique_ids,how='inner', on ='id' )

### This results in a basetable with unique project ids per row

In [135]:
final_df.shape

(28622, 19)

In [136]:
len(final_df['id'].unique())

28622

### Save dataframe to use for analysis

In [137]:
final_df.to_csv('kickstarter_data.csv',sep=';')